In [47]:
# get list of wordnet entities that need semantic universals
import json

with open("/content/drive/MyDrive/Colab Notebooks/llms4ol/geo/data/geonames_train.json", "r") as json_data:
    geonames_json = json.loads(json_data.read())
    json_data.close()

# get all wordnet entities
entities_list = [i['type'] for i in geonames_json]
# remove all duplicate entities
entities = list(set(entities_list))
len(entities)

660

In [48]:
# create entity dictionary to keep track of count of entity classes
# this ensures the dataset will be curated with balanced picks of entities
entity_count_dict = {}
for ent in entities:
  entity_count_dict[ent] = 0 # set initial count to 0 for each entity

print(entity_count_dict)

{'meander neck': 0, 'seat of government of a political entity': 0, 'military school': 0, 'mangrove island': 0, 'railroad stop': 0, 'intermittent lakes': 0, 'section of waterfall(s)': 0, 'marina': 0, 'snowfield': 0, 'radio observatory': 0, 'wreck': 0, 'trough': 0, 'blowout(s)': 0, 'rockfall': 0, 'distributary(-ies)': 0, 'reserve': 0, 'barracks': 0, 'slide': 0, 'harbor(s)': 0, 'business center': 0, 'historical third-order administrative division': 0, 'prison': 0, 'ruin(s)': 0, 'gasfield': 0, 'drainage basin': 0, 'monument': 0, 'quay': 0, 'cone(s)': 0, 'railroad yard': 0, 'gate': 0, 'butte(s)': 0, 'navigation canal(s)': 0, 'bridge': 0, 'seachannels': 0, 'salt pond': 0, 'landfill': 0, 'abandoned mission': 0, 'intermittent pond': 0, 'pinnacle': 0, 'continental rise': 0, 'administrative facility': 0, 'narrows': 0, 'section of independent political entity': 0, 'gulf': 0, 'facility': 0, 'cliff(s)': 0, 'novitiate': 0, 'arch': 0, 'seat of a fourth-order administrative division': 0, 'roadstead': 

In [49]:
print(min(entity_count_dict.items(), key=lambda x: x[1])[1])
print(max(entity_count_dict.items(), key=lambda x: x[1])[1])

0
0


In [52]:
from collections import Counter

date_counts = Counter(d['type'] for d in geonames_json)
#most_common = {'type': date_counts.most_common(1)[0][0]}
most_common = {'type': date_counts.most_common}
print("*** DATE COUNTS:",date_counts)
print("***************************************")
print("*** MOST COMMON:",most_common)

*** DATE COUNTS: Counter({'populated place': 2604140, 'stream': 563655, 'mountain': 291774, 'hill': 277280, 'hotel': 213687, 'farm': 207014, 'school': 179005, 'fourth-order administrative division': 165245, 'lake': 156441, 'intermittent stream': 143062, 'locality': 130141, 'building(s)': 126260, 'third-order administrative division': 117150, 'populated locality': 108699, 'church': 106999, 'island': 103584, 'section of populated place': 100473, 'well': 87255, 'valley': 77684, 'area': 66751, 'reservoir(s)': 66111, 'park': 66036, 'point': 64149, 'wadi': 64103, 'railroad station': 59983, 'cemetery': 56439, 'spring(s)': 52648, 'dam': 52055, 'pond': 50648, 'peak': 40123, 'abandoned populated place': 39846, 'bay': 39439, 'second-order administrative division': 38683, 'post office': 38314, 'canal': 37892, 'pass': 34708, 'marsh(es)': 33317, 'ridge(s)': 31583, 'bridge': 30777, 'historical fifth-order administrative division': 30447, 'homestead': 30377, 'house(s)': 30373, 'mine(s)': 30224, 'camp(

In [54]:
date_counts['stream']

563655

In [59]:
# curate dataset by picking 300000 values
geo_curated_dataset = []
print("** INITIAL CURATED DATASET LENGTH:", len(geo_curated_dataset))
# and picking samples based on under-represented entities based on min key values in entity dict
for ent in entities:
  print(ent,":",date_counts[ent])
  ent_curated_list = [i for i in geonames_json if i['type'] == ent]
  if date_counts[ent] < 100: # select all samples of entities represented less than 100 times
    geo_curated_dataset.extend(ent_curated_list)
  else:
    geo_curated_dataset.extend(ent_curated_list[:100]) # keep only 25 samples for these entities
  print("*** CURRENT DATASET LENGTH:", len(geo_curated_dataset))

print("*** FINAL CURATED DATASET LENGTH:", len(geo_curated_dataset))

** INITIAL CURATED DATASET LENGTH: 0
meander neck : 3
*** CURRENT DATASET LENGTH: 3
seat of government of a political entity : 17
*** CURRENT DATASET LENGTH: 20
military school : 204
*** CURRENT DATASET LENGTH: 120
mangrove island : 21
*** CURRENT DATASET LENGTH: 141
railroad stop : 7450
*** CURRENT DATASET LENGTH: 241
intermittent lakes : 1
*** CURRENT DATASET LENGTH: 242
section of waterfall(s) : 5
*** CURRENT DATASET LENGTH: 247
marina : 1478
*** CURRENT DATASET LENGTH: 347
snowfield : 87
*** CURRENT DATASET LENGTH: 434
radio observatory : 9
*** CURRENT DATASET LENGTH: 443
wreck : 102
*** CURRENT DATASET LENGTH: 543
trough : 195
*** CURRENT DATASET LENGTH: 643
blowout(s) : 547
*** CURRENT DATASET LENGTH: 743
rockfall : 7
*** CURRENT DATASET LENGTH: 750
distributary(-ies) : 3466
*** CURRENT DATASET LENGTH: 850
reserve : 9183
*** CURRENT DATASET LENGTH: 950
barracks : 490
*** CURRENT DATASET LENGTH: 1050
slide : 5
*** CURRENT DATASET LENGTH: 1055
harbor(s) : 4985
*** CURRENT DATASET L

In [60]:
# transform curated dataset to json format and save it
with open('geonames_train_curated.json', 'w') as fp:
    json.dump(geo_curated_dataset, fp)